# Loading data

In [1]:
import json

with open('clinais.train.json') as f:
    data = json.load(f)

from tqdm import tqdm

finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

100%|██████████| 781/781 [00:00<00:00, 9240.38it/s]


In [2]:
from datasets import load_from_disk

dataset = load_from_disk('augmented_dataset')

/home/joheras/.local/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


# Processing dataset

In [3]:
from transformers import AutoTokenizer

In [4]:
modelCheckpoint = "joheras/distilbert-base-spanish-uncased-finetuned-clinais"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2850
    })
    val: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
import evaluate

seqeval = evaluate.load("seqeval")

In [10]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [11]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at joheras/distilbert-base-spanish-uncased-finetuned-clinais were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at joheras/distilbert-base-spanish-uncased-finetuned-clinais and are newly initialized: 

In [12]:
training_args = TrainingArguments(
    output_dir="clinico-finetuned-augmented1",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/joheras/clinico-finetuned-augmented1 into local empty directory.
The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2850
  Num Epochs = 300
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 6900
  Number of trainable parameters = 66349070
Automatic

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.347083,0.001298,0.002222,0.001639,0.554787
2,No log,1.038590,0.012048,0.034444,0.017852,0.652679
3,No log,0.908844,0.040976,0.093333,0.056949,0.712460
4,No log,0.818948,0.060366,0.135556,0.083533,0.744559
5,No log,0.748396,0.094318,0.184444,0.124812,0.770426
6,No log,0.709403,0.113164,0.217778,0.148936,0.777158
7,No log,0.702202,0.116921,0.221111,0.152959,0.780039
8,No log,0.677576,0.157543,0.287778,0.203616,0.794413
9,No log,0.698344,0.149288,0.291111,0.197363,0.796355
10,No log,0.682146,0.177208,0.330000,0.230590,0.802086


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 128
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to clinico-finetuned-augmented1/checkpoint-23
Configuration saved in clinico-finetuned-augmented1/checkpoint-23/config.json
Model weights saved in clinico-finetuned-augmented1/checkpoint-23/pytorch_model.bin
tokenizer config file saved in clinico-finetuned-augmented1/checkpoint-23/tokenizer_config.js

TrainOutput(global_step=6900, training_loss=0.038896890805251355, metrics={'train_runtime': 6929.7647, 'train_samples_per_second': 123.381, 'train_steps_per_second': 0.996, 'total_flos': 1.1173263842304e+17, 'train_loss': 0.038896890805251355, 'epoch': 300.0})

In [13]:
trainer.push_to_hub()

Saving model checkpoint to clinico-finetuned-augmented1
Configuration saved in clinico-finetuned-augmented1/config.json
Model weights saved in clinico-finetuned-augmented1/pytorch_model.bin
tokenizer config file saved in clinico-finetuned-augmented1/tokenizer_config.json
Special tokens file saved in clinico-finetuned-augmented1/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

Upload file runs/Mar17_10-14-57_minion/events.out.tfevents.1679044506.minion.151280.0:  22%|##2       | 32.0k/…

To https://huggingface.co/joheras/clinico-finetuned-augmented1
   d31130a..6de2de2  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.34712482468443195}, {'name': 'Recall', 'type': 'recall', 'value': 0.55}, {'name': 'F1', 'type': 'f1', 'value': 0.42562338779019776}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8318667209469828}]}
To https://huggingface.co/joheras/clinico-finetuned-augmented1
   6de2de2..e33d632  main -> main



'https://huggingface.co/joheras/clinico-finetuned-augmented1/commit/6de2de2833a7c5011e1e068f8003fb307082befe'

In [14]:
!rm -rf clinico-finetuned-augmented1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
